
### *Note: This notebook was originally designed as a reference for building the agent. Several modifications were made to integrate the Langraph framework.*

In [1]:
!pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 2.3 MB/s eta 0:00:00


In [4]:
from langchain.chat_models import ChatOpenAI
import re
import requests

In [5]:
class LangraphAgent:
    def __init__(self, openai_api_key, spoonacular_api_key, custom_base_url):
        self.openai_api_key = openai_api_key
        self.spoonacular_api_key = spoonacular_api_key
        self.model = ChatOpenAI(
            model="gpt-4o-mini",
            temperature=0,
            openai_api_key=openai_api_key,
            base_url=custom_base_url,
        )

    def query_openai(self, query):
        try:
            response = self.model.invoke(query)
            return response  # Response already returns a clean result
        except Exception as e:
            return f"Error invoking model: {e}"

    def extract_ingredients(self, text):
        words = re.findall(r'\b\w+\b', text)
        query = f"Extract ingredients from the following text: '{words}'. Only give ingredients, don't give other things. If no ingredients are found just return None"
        try:
            response = self.model.invoke(query)
            ai_message_content = response.content
            print("ai_message_content:",ai_message_content)
            if(ai_message_content == "None"):
                ingredients = []
                # print("Ingredients not found")
            else:
            # Parse the response for ingredients
                ingredients = ai_message_content.split(", ")  # Assuming the response lists ingredients
                ingredients = [ingredient.strip() for ingredient in ingredients if ingredient]

        except Exception as e:
            return []
        return ingredients

    def get_recipe(self, ingredients):
        if not ingredients:
            return "No valid ingredients found to generate a recipe."
        url = f"https://api.spoonacular.com/recipes/findByIngredients?ingredients={','.join(ingredients)}&apiKey={self.spoonacular_api_key}"
        response = requests.get(url)

        if response.status_code == 200:
            recipes = response.json()
            if recipes:
                print(recipes)
                return [recipe['title'] for recipe in recipes]
            else:
                return "No recipes found for the given ingredients."
        else:
            return f"Error from Spoonacular API: {response.status_code}, {response.text}"

    def is_recipe_query(self, query):
        keywords = {'recipe', 'cook', 'ingredient', 'dish', 'food', 'meal'}
        return any(keyword in query.lower() for keyword in keywords)

    def handle_query(self, query):
        if self.is_recipe_query(query):
            ingredients = self.extract_ingredients(query)
            print(f"Extracted ingredients: {ingredients}")  # Debugging log
            if len(ingredients) == 0:
              return self.query_openai(query)
            else:
              return self.get_recipe(ingredients)
        else:
            return self.query_openai(query)

In [6]:
# Usage
openai_api_key = 'YOUR_OPENAI_API_KEY'  # Replace with your actual API key
spoonacular_api_key = 'YOUR_SPPONACULAR_API_KEY'  # Replace with your actual Spoonacular API key
custom_base_url = 'CUSTOM_OPENAI_BASE_URL'
agent = LangraphAgent(openai_api_key, spoonacular_api_key,custom_base_url)

response = agent.handle_query("I have tomato and potato as ingredients, suggest me some recipe.")
print(response)


<ipython-input-5-a4ab6c801fd8>:5: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  self.model = ChatOpenAI(


ai_message_content: tomato, potato
Extracted ingredients: ['tomato', 'potato']
[{'id': 655601, 'title': 'Penne with Sausage, Tomatoes and Potatoes', 'image': 'https://img.spoonacular.com/recipes/655601-312x231.jpg', 'imageType': 'jpg', 'usedIngredientCount': 2, 'missedIngredientCount': 3, 'missedIngredients': [{'id': 1001, 'amount': 0.5, 'unit': 'pound', 'unitLong': 'pounds', 'unitShort': 'lb', 'aisle': 'Milk, Eggs, Other Dairy', 'name': 'butter', 'original': '1/2 pound butter', 'originalName': 'butter', 'meta': [], 'image': 'https://img.spoonacular.com/ingredients_100x100/butter-sliced.jpg'}, {'id': 7036, 'amount': 4.0, 'unit': '', 'unitLong': '', 'unitShort': '', 'aisle': 'Meat', 'name': 'sausages', 'original': '4 sweet Italian sausages cut in chunks', 'originalName': 'sweet Italian sausages cut in chunks', 'meta': ['sweet', 'italian', 'cut in chunks'], 'extendedName': 'italian sweet sausages', 'image': 'https://img.spoonacular.com/ingredients_100x100/raw-pork-sausage.png'}, {'id': 1